In [2]:
x = 5
y = 6
z = x

# to let z be a reference to x instead of a copy of x, we can use the following
z = x
x = 7

print(x, y, z)

7 6 5


In [8]:
dictionary = {'a': 5, 'b': 2, 'c': 4}
sorted_dict = sorted(dictionary.items(), key=lambda x: x[1])
print(sorted_dict)
first_value = sorted_dict[0][1]
print(first_value)

[('b', 2), ('c', 4), ('a', 5)]
2


In [12]:
# for i in range(0,5):
#     print(i)

# now if we want to print in reverse order
for i in range(5,0,-1):
    print(i)

5
4
3
2
1


In [ ]:
 [<hnsw.node object at 0x00000177DF789E90>, <hnsw.node object at 0x00000177DF789F10>, <hnsw.node object at 0x00000177DF78A010>, <hnsw.node object at 0x00000177DF78A0D0>, <hnsw.node object at 0x00000177DF78A3D0>, <hnsw.node object at 0x00000177DF78A490>, <hnsw.node object at 0x00000177DF78A6D0>, <hnsw.node object at 0x00000177DF78A790>, <hnsw.node 
object at 0x00000177DF78A9D0>, <hnsw.node object at 0x00000177DF78AF10>, <hnsw.node object at 0x00000177DF78AD50>, <hnsw.node object at 0x00000177DF78B650>, <hnsw.node object at 0x00000177DF78BD10>, <hnsw.node object at 0x00000177DF78BC50>, <hnsw.node object at 0x00000177DF7A0210>, <hnsw.node object at 0x00000177DF7A0450>, <hnsw.node object at 0x00000177DF7A05D0>, <hnsw.node object at 0x00000177DF7A0650>, <hnsw.node object at 0x00000177DF7A06D0>, <hnsw.node object at 0x00000177DF7A0810>, <hnsw.node object at 0x00000177DF7A0E10>, <hnsw.node object at 0x00000177DF7A1050>, <hnsw.node object at 0x00000177DF7A1250>, <hnsw.node object at 0x00000177DF7A1C50>, <hnsw.node object at 0x00000177DF7A1ED0>, <hnsw.node object at 0x00000177DF7A2010>, <hnsw.node object at 0x00000177DF7A2150>, <hnsw.node object at 0x00000177DF7A2550>, <hnsw.node object at 0x00000177DF7A29D0>, <hnsw.node object at 0x00000177DF7A2C10>, <hnsw.node object at 0x00000177DF7A2E50>, <hnsw.node object at 0x00000177DF7A3650>, <hnsw.node object at 0x00000177DF7A36D0>, <hnsw.node object at 0x00000177DF7A3BD0>, <hnsw.node object at 0x00000177DF7A3E90>, <hnsw.node object at 0x00000177DF7A4150>, <hnsw.node object at 0x00000177DF7A4210>, <hnsw.node object at 0x00000177DF7A4450>, <hnsw.node object at 0x00000177DF7A47D0>, <hnsw.node object at 0x00000177DF7A4A50>, <hnsw.node object at 0x00000177DF7A4E10>, <hnsw.node object at 0x00000177DF7A4F10>, <hnsw.node object at 0x00000177DF7A5050>, <hnsw.node object at 0x00000177DF7A5110>, <hnsw.node object at 0x00000177DF7A5350>, <hnsw.node object at 0x00000177DF7A5410>, <hnsw.node object at 0x00000177DF7A53D0>, <hnsw.node object at 0x00000177DF7A5890>, <hnsw.node object at 0x00000177DF7A5E10>, <hnsw.node object at 0x00000177DF7A5E90>, <hnsw.node object at 0x00000177DF7A6010>, <hnsw.node object at 0x00000177DF7A6310>, <hnsw.node object at 0x00000177DF7A6390>, <hnsw.node object at 
0x00000177DF7A62D0>, <hnsw.node object at 0x00000177DF7A6710>, <hnsw.node object at 0x00000177DF7A6790>, <hnsw.node object at 0x00000177DF7A6A10>, <hnsw.node object at 0x00000177DF7A7210>, <hnsw.node object at 0x00000177DF7A7910>, <hnsw.node object at 0x00000177DF7A77D0>, <hnsw.node object at 0x00000177DF7A8390>, <hnsw.node object at 0x00000177DF7A84D0>, <hnsw.node object at 0x00000177DF7A8410>, <hnsw.node object at 0x00000177DF7A8750>, <hnsw.node object at 0x00000177DF7A8A10>, <hnsw.node object at 0x00000177DF7A8DD0>, <hnsw.node object at 0x00000177DF7A9010>, <hnsw.node object at 0x00000177DF7A9290>, <hnsw.node object at 0x00000177DF7A9350>, <hnsw.node object at 0x00000177DF7A9650>, <hnsw.node object at 0x00000177DF7A9750>, <hnsw.node object at 0x00000177DF7A99D0>, <hnsw.node object at 0x00000177DF7A9AD0>, <hnsw.node object at 0x00000177DF7A9CD0>, <hnsw.node object at 0x00000177DF7A9D50>]